## Combine NC Traffic and Police BJS datasets

In [ ]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [ ]:
dbname_police = 'police_bjs'
username = 'along528'
pswd = 'password'
con_police = None
con_police = psycopg2.connect(database = dbname_police, user = username, host='localhost', password=pswd)

In [ ]:
dbname_nctraffic = 'traffic_stops_nc'
username = 'along528'
pswd = 'password'
con_nctraffic = None
con_nctraffic = psycopg2.connect(database = dbname_nctraffic, user = username, host='localhost', password=pswd)

In [ ]:
sql_query = """
SELECT s.agency_id,s.agency_description,count(s.agency_id) FROM nc_stop s
GROUP BY s.agency_id,s.agency_description
ORDER BY s.agency_id
"""
nctraffic_agencies = pd.read_sql_query(sql_query,con_nctraffic)
#make agency description format same as in police bjs dataset
nctraffic_agencies['agency_description'] = map(str.upper,nctraffic_agencies['agency_description'])
nctraffic_agencies['agency_description'] = map(lambda x: x.replace(' ',''),nctraffic_agencies['agency_description'])
nctraffic_agencies['agency_description'] = map(lambda x: x.replace("'",""),nctraffic_agencies['agency_description'])
nctraffic_agencies['agency_description'] = map(lambda x: x.replace("DEPARTMENT","DEPT"),nctraffic_agencies['agency_description'])
nctraffic_agencies['agency_description'] = map(lambda x: x.replace("SHERIFFOFFICE","SHERIFFSOFFICE"),nctraffic_agencies['agency_description'])
nctraffic_agencies['agency_description'] = map(lambda x: x.replace("SHERIFFSOFFICE","SHERIFF"),nctraffic_agencies['agency_description'])
nctraffic_agencies

In [ ]:
sql_query = """
SELECT s."SURVEYID",s.agency,s.city,count(s."SURVEYID") FROM police_data_table s
WHERE s.state='NC'
GROUP BY s.agency,s.city,s."SURVEYID"
ORDER BY s.agency;
"""
policebjs_nc_agencies = pd.read_sql_query(sql_query,con_police)
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("'",""),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("DEPARTMENT","DEPT"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("SHERIFFOFFICE","SHERIFFSOFFICE"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("SHERIFFSOFFICE","SHERIFF"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("SHERIFFS","SHERIFF"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("HAVELOCK","HAVELOOK"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("NORTHCAROLINASTATEHWYPATROL","NCSTATEHIGHWAYPATROL"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies['agency'] = map(lambda x: x.replace("ROWAN","ROWANCOUNTY"),policebjs_nc_agencies['agency'])
policebjs_nc_agencies

In [ ]:
agencyid_map_police_to_traffic = {}
agencyid_map_traffic_to_police = {}
nmatch=0
for police_agency in policebjs_nc_agencies['agency']:
    match = False
    for nctraffic_agency in nctraffic_agencies['agency_description']:
        if nctraffic_agency==police_agency:
            nmatch+=1
            match=True
            break
    if match: 
            policeid = int(policebjs_nc_agencies[policebjs_nc_agencies['agency']==police_agency]['SURVEYID'])
            nctrafficid = int(nctraffic_agencies[nctraffic_agencies['agency_description']==police_agency]['agency_id'])
            agencyid_map_police_to_traffic[policeid] = nctrafficid
            agencyid_map_traffic_to_police[nctrafficid] = policeid
print "Number of matched agencies =",nmatch
agencyid_map_traffic_to_police

In [ ]:
#save maps to be used elsewhere
import pickle
pickle.dump( agencyid_map_police_to_traffic, open( "agencyid_map_police_to_traffic.p", "wb" ) )
pickle.dump( agencyid_map_traffic_to_police, open( "agencyid_map_traffic_to_police.p", "wb" ) )

In [2]:
#or load them from before
import pickle
agencyid_map_police_to_traffic = pickle.load( open( "agencyid_map_police_to_traffic.p", "rb" ) )
agencyid_map_traffic_to_police = pickle.load( open( "agencyid_map_traffic_to_police.p", "rb" ) )
print len(agencyid_map_traffic_to_police)

88


Now get traffic stops dataframe for just matched police departments

In [ ]:
#rather than loading whole dataset I should wait to see if I can develop a query to condense each agency 
#down to one entry with one or a few versions of the RPSI
sql_query = """
SELECT * FROM nc_stop s
WHERE """
isFirst = True
for key in agencyid_map_traffic_to_police:
    if isFirst:
        isFirst = False
    else: 
        sql_query += " OR "
    sql_query += " s.agency_id = %d \n" % (key)
sql_query+=";"
nctraffic_stops_matched = pd.read_sql_query(sql_query,con_nctraffic)
nctraffic_stops_matched